# Example: Properties of Markov Models and Stationary Distributions
The objective of this example is to familiarize students with Markov models and components of Markov models, in particular the transition matrix $\mathbf{P}$, computing the stationary distribution $\pi$, and finally sampling the stationary distribution using categorical distribution.

### Background
A discrete-time Markov chain is a sequence of random variables $X_{1},\dotsc,X_{n}$ with 
the [Markov property](https://en.wikipedia.org/wiki/Markov_property), 
i.e., the probability of moving to the next state depends only on the present and not past states:

$$
\begin{equation*}
P(X_{n+1} = x | X_{1}=x_{1}, \dots, X_{n}=x_{n}) = P(X_{n+1} = x | X_{n}=y)
\end{equation*}
$$

For finite state spaces $\mathcal{S}$, the probability of moving from the state(s) $i\rightarrow{j}$ in the next step, 
is encoded in the transition matrix $p_{ij}\in\mathbf{P}$: 

$$
\begin{equation*}
p_{ij} = P(X_{n+1}~=~j~|~X_{n}~=~i)
\end{equation*}
$$

The transition matrix $\mathbf{P}$ has interesting properties: 
* First, the rows of transition matrix $\mathbf{P}$ sum to unity, i.e., each row encodes the probability of all possible outcomes. 
Thus, it must sum to one.
* Second, if the transition matrix  $\mathbf{P}$ is invariant, $p_{ij}$ doesn't change as $n\rightarrow{n+1}~\forall{n}$.

## Setup
Let's load some packages that are required for the example by calling the `include(...)` function on our initialization file `Include.jl`:

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-13/L13a`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-13/L13a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-13/L13a/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-13/L13a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-13/L13a/Manifest.toml`


```julia
iterate(P::Array{Float64,2}, counter::Int; 
        maxcount::Int = 100, ϵ::Float64 = 0.1) -> Array{Float64,2}
```
> Recursively computes a stationary distribution. Computation stops if ||P_new - P|| < ϵ or the max number of iterations is hit. 

In [2]:
function iterate(P::Array{Float64,2}, counter::Int; 
        maxcount::Int = 100, ϵ::Float64 = 0.1)::Array{Float64,2}

    # base case -
    if (counter == maxcount)
        return P
    else
        # generate a new P -
        P_new = P^(counter+1)
        err = P_new - P;
        if (norm(err)<=ϵ)
            return P_new
        else
            # we have NOT hit the error target, or the max iterations
            iterate(P_new, (counter+1), maxcount=maxcount, ϵ = ϵ)
        end
    end
end;

#### Constants
In the simulations below, we'll need some constant values that we set here. In particular, we set a value for the `number_of_hidden_states` variable, the `number_of_simulation_steps` variable (the number of steps that we take in a Markov Chain), and the `number_of_samples ` variable:

In [3]:
number_of_hidden_states = 3;
number_of_simulation_steps = 10000;
number_of_samples = 10000;

## Setup transition matrix for a three-state Markov Model
Let's make these ideas more concrete by looking at a simple three-state example:

<div>
    <center>
        <img src="figs/Fig-ThreeState-MM-Schematic.svg" width="580"/>
    </center>
</div>

In this example we have `three` states $\mathcal{S}=\left\{1,2,3\right\}$ and the probability of moving between state $i\rightarrow{j}$ in the next time step, denoted as $p_{ij}$, are elements of the matrix $\mathbf{P} \in \mathbb{R}^{3\times{3}}$.

In [4]:
P = [
    0.05 0.95 0.0 ; # moves for state 1
    0.6 0.2 0.2 ; # moves for state 2
    0.0 0.3 0.7 ; # moves for state 3
];

In [5]:
rank(P)

3

### Check: do the rows of the transition matrix $\mathbf{P}$ sum to `1`?
We know that the rows of the transition matrix $\mathbf{P}$ must sum to `1`, i.e., if we are in state $s_{i}\in\mathcal{S}$ at time $t$, then at time $t+1$ we have to be in $s_{j}\in\mathcal{S}$. 
* Let's check if the transition matrix $\mathbf{P}$ meets this criteria using the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) by iterating over the rows of the transition matrix $\mathbf{P}$ and checking the sum of each row. If any row does not meet this criterion, an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) will be thrown.

In [6]:
for i ∈ 1:number_of_hidden_states
    @assert sum(P[i,:]) == 1
end

## Compute the stationary distribution $\pi$

For a non-periodic Markov chain with a finite state space $\mathcal{S}$ and an invariant state transition matrix $\mathbf{P}$,
the state vector at time $j$, denoted by $\mathbf{\pi}_{j}$, has the property:

$$
\begin{equation*}
\sum_{s\in\mathcal{S}}\pi_{sj} = 1\qquad\forall{j}
\end{equation*}
$$

where $\pi_{sj}\geq{0},\forall{s}\in\mathcal{S}$. The state of the Markov chain at time step $n+1$, denoted by $\mathbf{\pi}_{n+1}$, is given by:

$$
\begin{equation*}
\mathbf{\pi}_{n+1} = \mathbf{\pi}_{1}\cdot\left(\mathbf{P}\right)^n
\end{equation*}
$$

where $\mathbf{\pi}_{n+1}$ is the state vector at time step $n+1$ and $\left(\mathbf{P}\right)^n$ is the transition matrix raised to the $n$th power. Finally, a unique stationary distribution $\pi$ exists, where $\mathbf{P}^{k}$ converges to a `rank-one` matrix in which each row is the stationary distribution $\pi$:

$$
\begin{equation*}
\lim_{k\rightarrow\infty} \mathbf{P}^{k} = \mathbf{1}^{T}\otimes\pi
\end{equation*} 
$$

where $\mathbf{1}$ is a column vector of all 1s. The operator $\otimes$ denotes a __Left Matrix Vector Product operation__.

#### Implementation details
We'll compute the stationary distribution $\pi$ using the recursive `iterate(...)` method. During each call to the `iterate(...)` method, we compute the matrix power of transition matrix $\mathbf{P}$. We continue to call the `iterate(...)` method until we hit one of two possible conditions:

* The `base case` for the recursion occurs when the `counter == maxcount`, at this point the recursion stops, and the matrix $\mathbf{P}$ is returned
* The recursion also stops when the difference between subsequent powers of the matrix $\mathbf{P}$ is smaller than a specified threshold

In [7]:
π̄ = iterate(P,1, ϵ = 0.0000001)

3×3 Matrix{Float64}:
 0.274809  0.435115  0.290076
 0.274809  0.435115  0.290076
 0.274809  0.435115  0.290076

### Check: Is the rank condition on the stationary distribution $\bar{\pi}$ correct?
Once we reach the stationary distribution, the rank of the stationary distribution $\pi$ should be equal to `1`. Let's check whether this condition is true using the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert). 
* If we do not meet this criterion, an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) will be thrown, and we should try to use more iterations or a tighter numerical tolerance value for $\epsilon$. We'll compute the rank using the [rank function](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.rank) which is exported by the [Julia Statistics package](https://docs.julialang.org/en/v1/stdlib/Statistics/#Statistics)

In [8]:
@assert rank(π̄) == 1

### Compute Markov Chain Distribution $\pi$ using the Left Matrix Vector Product
We can also approach the computation of the stationary distribution $\bar{\pi}$ by directly iterating the expression:

$$
\pi_{n+1} = \pi_{n}\cdot\mathbf{P}\quad\,n=1,2,\dots
$$

As $n\rightarrow\infty$, i.e., as we do more iterations, the difference between subsequent iterations becomes small $||\pi_{n+1}-\pi_{n}||<\epsilon$ for a non-periodic Markov chain, where $\pi_{n}\rightarrow\bar{\pi}$ as $n\rightarrow\infty$. 

In [9]:
π₁ = [0.0,1.0,0.0]; # initial state = state 2
direct_state_distribution = Dict{Int,Array{Float64,1}}();
direct_state_distribution[1] = π₁;
for n = 2:number_of_simulation_steps
    πᵢ = transpose(π₁)*(P)^(n-1)
    direct_state_distribution[n] =  transpose(πᵢ)
end
foreach(i-> println(direct_state_distribution[i]), 1:20);

[0.0, 1.0, 0.0]
[0.6, 0.2, 0.2]
[0.15, 0.6699999999999999, 0.18]
[0.4094999999999999, 0.33049999999999996, 0.26]
[0.21877499999999994, 0.533125, 0.2481]
[0.3308137499999999, 0.38889124999999997, 0.28029499999999996]
[0.2498754374999999, 0.4761398124999999, 0.27398475]
[0.2981776593749999, 0.4148050531249999, 0.28701728749999994]
[0.26379191484374986, 0.45233497328124994, 0.283873111875]
[0.2845905797109374, 0.4262312473203125, 0.2891781729687499]
[0.2699682773777342, 0.442360752080078, 0.28767097054218743]
[0.2789148651169334, 0.4312433050875194, 0.2898418297955468]
[0.27269172630835825, 0.43817033181725473, 0.2891379418743867]
[0.2765367854057707, 0.4334325889187073, 0.29003062567552157]
[0.2738863926215128, 0.43640565162188016, 0.28970795575660657]
[0.2755377106042036, 0.4343855900417952, 0.2900766993540006]
[0.27440823955528726, 0.4356609528885528, 0.2899308075561595]
[0.275116983710896, 0.43479926042208134, 0.2900837558670222]
[0.2746354054387935, 0.43534611336987405, 0.29001848119

## Compute a Sequence of states $s_{i}\in\mathcal{S}$ from our Markov Model using a Categorical Distribution
We can get the dynamics predicted by the [Markov model](https://en.wikipedia.org/wiki/Markov_model), i.e., the sequence of states and state transitions by sampling the transition probability matrix $\mathbf{P}$ directly. 
* Now that we are sure that the transition matrix $\mathbf{P}$ is properly formulated let's populate the `hidden_state_probability_dictionary,` which holds the [categorical distribution](https://en.wikipedia.org/wiki/Categorical_distribution) modeling the transition probability for each hidden state $s\in\mathcal{S}$, i.e., the probability that we transition from state $i\rightarrow{j}$ in the next time step

In [10]:
hidden_state_probability_dictionary = Dict{Int,Categorical}();
for i ∈ 1:number_of_hidden_states
    hidden_state_probability_dictionary[i] = Categorical(P[i,:])
end
hidden_state_probability_dictionary

Dict{Int64, Categorical{P} where P<:Real} with 3 entries:
  2 => Categorical{Float64, Vector{Float64}}(support=Base.OneTo(3), p=[0.6, 0.2…
  3 => Categorical{Float64, Vector{Float64}}(support=Base.OneTo(3), p=[0.0, 0.3…
  1 => Categorical{Float64, Vector{Float64}}(support=Base.OneTo(3), p=[0.05, 0.…

In [11]:
d_test = hidden_state_probability_dictionary[1]

Categorical{Float64, Vector{Float64}}(support=Base.OneTo(3), p=[0.05, 0.95, 0.0])

In [12]:
rand(d_test,100) |> x-> findall(x == 2,x) |> length

LoadError: MethodError: no method matching findall(::Bool, ::Vector{Int64})

[0mClosest candidates are:
[0m  findall(::Bool)
[0m[90m   @[39m [90mBase[39m [90m[4marray.jl:2507[24m[39m
[0m  findall([91m::Base.Fix2{typeof(in), <:Union{Real, Array{<:Real}}}[39m, ::Array{<:Real})
[0m[90m   @[39m [90mBase[39m [90m[4marray.jl:2608[24m[39m
[0m  findall([91m::Base.Fix2{typeof(in)}[39m, ::AbstractArray)
[0m[90m   @[39m [90mBase[39m [90m[4marray.jl:2617[24m[39m
[0m  ...


Now, we generate `number_of_simulation_steps` worth of dynamic data by sampling the `hidden_state_probability_dictionary.` We store these simulation results in the `hidden_simulation_dict` dictionary, where the `key` holds the time index and the `value` is the system's state, i.e., $s_{i}\in\mathcal{S}$.
* We start by specifying an initial state `sᵢ = 1`. At each iteration of the loop, we pull out the [categorical distribution](https://en.wikipedia.org/wiki/Categorical_distribution) corresponding to the state, i.e., the row in the transition matrix $\mathbf{P}$ corresponding to state $s_{i}$. We generate the state at the next step by drawing a sample using the `rand(...)` function.

In [13]:
hidden_simulation_dict = Dict{Int,Int}();
sᵢ = 1; # harcode the start state: we could draw from the stationary distribuition
hidden_simulation_dict[1] = sᵢ;
for i ∈ 2:number_of_simulation_steps

    # get the categorical distribution for sᵢ 
    sᵢ = hidden_state_probability_dictionary[sᵢ] |> d -> rand(d);
    
    # capture -
    hidden_simulation_dict[i] = sᵢ
end
foreach(i -> println("Soln: (t=$(i),s=$(hidden_simulation_dict[i]))"), 1:20)

Soln: (t=1,s=1)
Soln: (t=2,s=2)
Soln: (t=3,s=1)
Soln: (t=4,s=2)
Soln: (t=5,s=1)
Soln: (t=6,s=2)
Soln: (t=7,s=2)
Soln: (t=8,s=1)
Soln: (t=9,s=2)
Soln: (t=10,s=1)
Soln: (t=11,s=2)
Soln: (t=12,s=1)
Soln: (t=13,s=2)
Soln: (t=14,s=1)
Soln: (t=15,s=2)
Soln: (t=16,s=1)
Soln: (t=17,s=2)
Soln: (t=18,s=2)
Soln: (t=19,s=1)
Soln: (t=20,s=2)


### Check: Do we recover the stationary distribution $\bar{\pi}$?
Just like a dynamic system, e.g., concentration balances in a steady-state reactor, if we wait long enough, the system should approach a steady state (assuming the steady state exists and is stable). By analogy, if we take enough time steps for our random dynamic system, the distribution of states should follow the stationary distribution $\bar{\pi}$.

In [14]:
S = [hidden_simulation_dict[i] for i ∈ 1:number_of_simulation_steps];
NS₁ = findall(x-> x == 1, S) |> length;
NS₂ = findall(x-> x == 2, S) |> length;
NS₃ = findall(x-> x == 3, S) |> length;

Finally, we can compute the relative frequency (probability) of the states `s = 1`, `s = 2` and `s = 3`:

In [15]:
PS1 = NS₁/number_of_simulation_steps;
PS2 = NS₂/number_of_simulation_steps;
PS3 = NS₃/number_of_simulation_steps;
println("The sampled probability (p₁,p₂,p₃) = ($(PS1), $(PS2), $(PS3))")

The sampled probability (p₁,p₂,p₃) = (0.2712, 0.435, 0.2938)


## What happens if we sample $\bar{\pi}$ directly?
Suppose we don't care about the dynamics (time behavior) of our three-state random system. Instead, we are only interested in the steady-state behavior, i.e., the stationary distribution. We can sample $\bar{\pi}$ directly, but if we do that, we lose all the information about the transitions, i.e., all the time behavior.
* Let's create a [categorical distribution](https://en.wikipedia.org/wiki/Categorical_distribution) using the stationary probability of our Markov chain using the [Distributions.jl](https://github.com/JuliaStats/Distributions.jl) package, save this distribution in the variable `d`:

In [16]:
d = Categorical(π̄[1,:]);

We generate samples from the [categorical distribution](https://en.wikipedia.org/wiki/Categorical_distribution) saved in the distribution `d` using the [rand function](https://docs.julialang.org/en/v1/stdlib/Random/#Base.rand). This allows us to `simulate` the system encoded by our three-state Markov model. For example:

In [17]:
rand(d,100)

100-element Vector{Int64}:
 2
 2
 2
 3
 2
 3
 1
 2
 2
 1
 2
 2
 2
 ⋮
 3
 2
 3
 1
 2
 2
 2
 2
 3
 1
 1
 2

### Check: Do we recover the stationary distribution $\bar{\pi}$?
If the distribution `d` represents the stationary distribution $\bar{\pi}$, we should be able to generate many samples and estimate the probability that we are in state `s=1`, `s=2`, or `s=3`. Let's sample the distribution `d` to recover the stationary distribution $\pi$. 
* We'll compute `number_of_samples` from the distribution `d` and then calculate the frequency of `s = 1`, `s = 2`, and `s = 3` values. These should converge to the stationary probability as the `number_of_samples` becomes large:

In [18]:
samples = rand(d, number_of_samples);

Compute the counts of the states `s = 1`, `s = 2` and `s = 3` from the `samples` array:

In [19]:
N₁ = findall(x-> x == 1, samples) |> length;
N₂ = findall(x-> x == 2, samples) |> length;
N₃ = findall(x-> x == 3, samples) |> length;

Finally, we can compute the relative frequency (probability) of the states `s = 1`, `s = 2` and `s = 3` 

In [20]:
p̂₁ = N₁/number_of_samples;
p̂₂ = N₂/number_of_samples;
p̂₃ = N₃/number_of_samples;
println("The sampled probability (p₁,p₂,p₃) = ($(p̂₁), $(p̂₂), $(p̂₃))")

The sampled probability (p₁,p₂,p₃) = (0.2705, 0.4321, 0.2974)
